# YOLOv10 Garbage Detection – Fine-Tuning (v2)

This notebook performs fine-tuning of a YOLOv10 object detection model for garbage
classification using a previously trained custom checkpoint.

**Key Highlights**
- Model: YOLOv10
- Task: Object Detection
- Training Type: Fine-tuning from custom checkpoint (v1)
- Dataset: Garbage Dataset (YOLO format)
- Output: Optimized detection model (`best.pt`)

## Environment Setup

This section sets up the YOLOv10 environment and installs the required dependencies.
The training process is executed on Google Colab with GPU acceleration.

In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10
!pip install -r requirements.txt

Cloning into 'yolov10'...
remote: Enumerating objects: 20338, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20338 (delta 2), reused 0 (delta 0), pack-reused 20334 (from 2)
Receiving objects: 100% (20338/20338), 11.10 MiB | 11.77 MiB/s, done.
Resolving deltas: 100% (14354/14354), done.
/content/yolov10
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.0.1


In [ ]:
!pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 129.9 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [ ]:
import torch

print("PyTorch version: ", torch.__version__)

2.9.0+cu126


## Dataset

The dataset contains labeled garbage images with the following classes:
- Plastic
- Paper
- Metal
- Glass
- Organic

The dataset is provided in YOLO format and split into training and validation sets.

In [ ]:
!unzip Trash.v2i.yolov8.zip

Archive:  Trash.v2i.yolov8.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/32911398817_e6ef283545_o_jpg.rf.d123995b544925df2330c03c82ee2f47.jpg  
 extracting: test/images/32911436277_8e046cc764_o_jpg.rf.0c4bb4717c22a09859b75c85f0c60e7c.jpg  
 extracting: test/images/32912101117_743a26fa9f_o_jpg.rf.501efaaa83d69ae75088e39856b3b56f.jpg  
 extracting: test/images/32912196167_9db352d672_o_jpg.rf.3dae88cd57b8a669bc6910e24461bde5.jpg  
 extracting: test/images/33978204438_b8c0c2b2c9_o_jpg.rf.69a24f953a6e8f71d4cabcd2856ae76d.jpg  
 extracting: test/images/33978213438_74b185f6e1_o_jpg.rf.41cdc015ddaa1aa3866304468c7e04b1.jpg  
 extracting: test/images/33978471308_9bc844ac9f_o_jpg.rf.642b77e6f066e1da14ed3f43558b3985.jpg  
 extracting: test/images/40888879023_df1db6fbd9_o_jpg.rf.d6ae123ef7edadb8e8641e22f241f761.jpg  
 extracting: test/images/40889050463_4528c

## Training Strategy

This experiment applies fine-tuning using a previously trained YOLOv10 checkpoint
(`best.pt`) from version 1. Fine-tuning enables the model to retain learned features
while adapting to additional data, resulting in faster convergence and improved
detection performance.

In [ ]:
!yolo task=detect \
  mode=train \
  model=/content/best.pt \
  data=/content/data.yaml \
  epochs=100 \
  imgsz=640 \
  batch=16 \
  device=0 \
  name=yolov10_garbage_v2

Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10_garbage_v2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

## Inference

The fine-tuned model is evaluated on unseen images to verify its detection performance.
A confidence threshold of 0.25 is used during inference.

In [ ]:
!yolo task=detect \
  mode=predict \
  model=/content/yolov10/runs/detect/yolov10_garbage_v2/weights/best.pt \
  source=/content/6H2ugVf6pSVopBUDOOiddQwxmFcCTdTSPQfQGl4j.jpeg \
  conf=0.25

Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10s summary (fused): 106 layers, 7,219,935 parameters, 0 gradients, 21.4 GFLOPs

image 1/1 /content/6H2ugVf6pSVopBUDOOiddQwxmFcCTdTSPQfQGl4j.jpeg: 640x480 1 plastic, 45.1ms
Speed: 2.5ms preprocess, 45.1ms inference, 16.2ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/yolov10/runs/detect/predict5
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!zip -r yolov10.zip /content/yolov10

  adding: content/yolov10/ (stored 0%)
  adding: content/yolov10/.git/ (stored 0%)
  adding: content/yolov10/.git/config (deflated 31%)
  adding: content/yolov10/.git/objects/ (stored 0%)
  adding: content/yolov10/.git/objects/pack/ (stored 0%)
  adding: content/yolov10/.git/objects/pack/pack-d4d8290e736e6b95d6698beed9556f43a8dfd5e8.pack (deflated 1%)
  adding: content/yolov10/.git/objects/pack/pack-d4d8290e736e6b95d6698beed9556f43a8dfd5e8.idx (deflated 2%)
  adding: content/yolov10/.git/objects/info/ (stored 0%)
  adding: content/yolov10/.git/index (deflated 60%)
  adding: content/yolov10/.git/refs/ (stored 0%)
  adding: content/yolov10/.git/refs/remotes/ (stored 0%)
  adding: content/yolov10/.git/refs/remotes/origin/ (stored 0%)
  adding: content/yolov10/.git/refs/remotes/origin/HEAD (stored 0%)
  adding: content/yolov10/.git/refs/heads/ (stored 0%)
  adding: content/yolov10/.git/refs/heads/main (stored 0%)
  adding: content/yolov10/.git/refs/tags/ (stored 0%)
  adding: content/yolov

## Conclusion

The fine-tuned YOLOv10 model demonstrates improved performance compared to the initial
version. Further improvements may be achieved by expanding the dataset, applying
additional data augmentation, or tuning training hyperparameters.